In [1]:
!pip install trimesh openpyxl open3d

In [2]:
import pandas as pd
import trimesh
import glob
import os
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
att_path = "/kaggle/input/3dattributes" # thư mục chứa file get-data.xlsx
obj_path = "kaggle/input/3dobject/3d model" # thư mục chứ file .obj

In [94]:
data = pd.read_excel(f"{att_path}/get-data.xlsx", sheet_name=None)

In [96]:
samples = {} # biến chứa dữ liệu của tất cả các object, mỗi object chứa 3 thông tin
for sheet in tqdm(data.keys(), position=0):
    obj_id = sheet.split("Sheet")[-1].strip() # lấy object id từ tên sheet
    try:
        obj = trimesh.load_mesh(f"{obj_path}/{obj_id}.obj", process=False)
    except:
        obj = trimesh.load_mesh(f"{obj_path}/{obj_id}.OBJ", process=False) # có một obj có đuôi là OBJ mà em lỡ up lên kaggle rồi nên phải try except :v, thầy đổi cái đuôi của file .OBJ thành .obj thì khỏi bỏ cái try except này cũng đc
    atts = data[sheet].dropna(how="all").dropna(axis=1, how="all").iloc[2:] # tất hàng và cột chứa tất cả giá trị là Nan, lấy từ hàm thứ 2 đổ đi vì 2 hàng đầu là "Input" và tên cột
    atts.drop(atts.columns[0], axis=1, inplace=True) # bỏ cột STT
    atts.columns = ["extrution_width", "layer_height", "speed", "infill", "time", "length", "weight"] # đặt lại tên cột
    atts.reset_index(drop=True, inplace=True) # đặt lại index
    atts["volume"] = obj.volume # thêm cột volume
    atts["area"] = obj.area # thêm cột area
    samples[obj_id] = { # mỗi object có 3 thông tin: vertices, attributes(4 cột đâu của file xlsx và 2 cột thông số của object), nhãn (3 cột cuối của file xlsx)
        "obj": obj,
        "atts": atts[["extrution_width", "layer_height", "speed", "infill", "volume", "area"]],
        "targets": atts[["time","length","weight"]]
    }
samples[list(samples.keys())[0]]["obj"].show()

In [91]:
samples[list(samples.keys())[64]]["atts"] 

In [ ]:
samples[list(samples.keys())[64]]["targets"] 

**Reduce vertices**

In [100]:
# tìm số faces nhỏ nhất trong tất cả object
min_faces = len(samples["1"]["obj"].faces)
for obj_id in samples.keys():
    n_faces = len(samples[obj_id]["obj"].faces)
    if n_faces < min_faces:
        min_faces = n_faces
print("Min n_faces: ", min_faces)

# giảm face của tất cả object về bằng số face nhỏ nhất và tìm số vertices nhỏ nhất, 
# vì sau khi giảm face của object thì độ chênh lệch vertice giữa các object chỉ vài cái nên tìm số v nhỏ nhất để những cái nào nhiều v hơn thì sẽ bỏ cho bằng v nhỏ nhất
min_vertices = len(samples["1"]["obj"].vertices)
for obj_id in samples.keys():
    samples[obj_id]["obj"] = samples[obj_id]["obj"].simplify_quadratic_decimation(min_faces) # hàm giảm face
    n_vertices = len(samples[obj_id]["obj"].vertices)
    if n_vertices < min_vertices:
        min_vertices = n_vertices
print("Min n_vertices", min_vertices)

# giảm v của mọi obj về bằng v nhỏ nhất
for obj_id in samples.keys():
    mask = np.full(len(samples[obj_id]["obj"].vertices), True) # tạo mask
    remove = len(samples[obj_id]["obj"].vertices) - min_vertices # số lượng v của obj hiện tại cần bỏ để bằng với số v nhỏ nhất
    if remove > 0: 
        mask[-remove:] = False # bỏ lượng chênh lệch ở cuối
        samples[obj_id]["obj"].update_vertices(mask) 

**Preprocessing**

In [101]:
# lưu 3 thông tin của tất cả obj thành 3 list để tiền xử lý
vertices = []
attributes = []
targets = []
for obj_id in samples.keys():
    vertices.append(samples[obj_id]["obj"].vertices) # chỉ lấy vertices
    attributes.append(samples[obj_id]["atts"])
    targets.append(samples[obj_id]["targets"])
    

In [102]:
from sklearn.model_selection import train_test_split

v_train, v_test, a_train, a_test, y_train, y_test = train_test_split(vertices, attributes, targets, test_size=0.2) # chia train test cho 3 thông tin


In [103]:
from sklearn.preprocessing import MinMaxScaler

# vì mỗi obj có nhiều vertices(v) và nhiều attributes(a), mỗi v và mỗi a lại có nhiều cột, 
# nên để scale tất cả dữ liệu theo từng cột thì ta phải nối từng cột của tất cả obj lại với nhau sau đó thực hiện scale

print(np.array(v_train).shape)
print(np.array(a_train).shape)
print(np.array(y_train).shape)

# mỗi phần tử có nhiều v nên nếu nối cột lại với nhau thì v sẽ liên tiếp nhau, nên giữ lại số lượng v của 1 obj để sau khi scale xong ta sẽ chia lại v cho mỗi obj
len_v = len(v_train[0])
len_a = len(a_train[0]) # tương tự với a, mỗi obj chỉ có 1 y nên ko cần nhớ số lượng y

v_train_sc = []
a_train_sc = pd.DataFrame()
y_train_sc = pd.DataFrame() # nối y luôn để scale

# nối tập train
for i in range(len(v_train)):
    v_train_sc.extend(v_train[i])
    a_train_sc = pd.concat([a_train_sc, a_train[i]])
    y_train_sc = pd.concat([y_train_sc, y_train[i]])

# nối tập test
v_test_sc = []
a_test_sc = pd.DataFrame()
y_test_sc = pd.DataFrame()
for i in range(len(v_test)):
    v_test_sc.extend(v_test[i])
    a_test_sc = pd.concat([a_test_sc, a_test[i]])
    y_test_sc = pd.concat([y_test_sc, y_test[i]])

# fit và scale tập train sau đó dùng scale đó scale cho tập test
v_scaler = MinMaxScaler()
v_train_sc = v_scaler.fit_transform(v_train_sc)
v_test_sc = v_scaler.transform(v_test_sc)

# tương tự cho a
a_scaler = MinMaxScaler()
a_train_sc = a_scaler.fit_transform(a_train_sc)
a_test_sc = a_scaler.transform(a_test_sc)

# tương tự cho y
y_scaler = MinMaxScaler()
y_train_sc = y_scaler.fit_transform(y_train_sc)
y_test_sc = y_scaler.transform(y_test_sc)

for i in range(len(v_train.copy())):
    v_train[i] = v_train_sc[i*len_v:(i+1)*len_v]
    a_train[i] = a_train_sc[i*len_a:(i+1)*len_a]
    y_train[i] = y_train_sc[i:i+1]

# chia lại v,a,y cho mỗi obj
for i in range(len(v_test.copy())):
    v_test[i] = v_test_sc[i*len_v:(i+1)*len_v]
    a_test[i] = a_test_sc[i*len_a:(i+1)*len_a]
    y_test[i] = y_test_sc[i:i+1]

y_train = np.array(y_train).squeeze() # y dư 1 dim nên giảm dim
v_train = np.array(v_train)
a_train = np.array(a_train)

y_test = np.array(y_test).squeeze() # y dư 1 dim nên giảm dim
v_test = np.array(v_test)
a_test = np.array(a_test)

# kiểm tra lại shape xem có bằng với ban đầu chưa
print(v_train.shape)
print(a_train.shape)
print(y_train.shape)

In [104]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *

In [105]:
obj_inputs = Input(shape=v_train[0].shape)
att_inputs = Input(shape=a_train[0].shape)

# CNN cho v
nodes = Conv1D(32, kernel_size=3)(obj_inputs)
nodes = MaxPooling1D()(nodes)
nodes = SpatialDropout1D(0.1)(nodes)
nodes = Flatten()(nodes)
obj_outputs = Dense(5)(nodes) # output shape (,5)

# CNN cho a, không nối được output của v với a vì shape của output v là (, 5) còn shape của a là (,6,32) (6 cột 32 hàng) nên tạm thời em bỏ a vào 1 cái cnn y chang cnn của v luôn
nodes = Conv1D(32, kernel_size=3)(att_inputs)
nodes = MaxPooling1D()(nodes)
nodes = SpatialDropout1D(0.1)(nodes)
nodes = Flatten()(nodes)
att_outputs = Dense(5)(nodes) # output shape (,5)

# gộp 2 output lại thành 1 input
total_inputs = Concatenate()([obj_outputs, att_outputs])

nodes = Dense(64)(total_inputs)
nodes = Dropout(0.1)(nodes)
outputs = Dense(3)(nodes)

model = Model(inputs=[obj_inputs, att_inputs], outputs=outputs)
model.compile(loss="mae", optimizer=Adam())
model.summary()

In [109]:
model.fit(x=[v_train, a_train], y=y_train, epochs=10, batch_size=4)

In [108]:
import matplotlib.pyplot as plt 

plt.plot(model.history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [112]:
preds = model.predict([v_test, a_test])

In [126]:
y_test

In [125]:
preds

In [124]:
np.mean(np.abs(preds - y_test) / y_test * 100) # độ khác nhau trung bình (%)